In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
all_drives = pd.read_csv('../data/espn_drives2009-2017.csv')

In [3]:
all_drives.head(5)

,Unnamed: 0,away,away_score_after,away_score_before,drive,home,home_score_after,home_score_before,offense,plays,result,time,yds_gained,gameId
0,0,PIT,0,0,0,TEN,0,0,PIT,3,Punt,1:44,2,290910023
1,1,PIT,0,0,1,TEN,0,0,TEN,3,Punt,1:52,2,290910023
2,2,PIT,0,0,2,TEN,0,0,PIT,5,Punt,3:04,2,290910023
3,3,PIT,0,0,3,TEN,0,0,TEN,6,Missed FG,1:35,38,290910023
4,4,PIT,0,0,4,TEN,0,0,PIT,3,Punt,1:55,-6,290910023


In [4]:
all_drives['result'].unique()

array(['Punt', 'Missed FG', 'Intercepted Pass', 'Touchdown', 'Fumble',
       'Field Goal', 'End of Half', 'End of Game', 'Downs',
       'End of 1st Half', 'Joshua Cribbs 67 Yard Punt Return',
       'Adrian Peterson 64 Yard Run', 'JAC Timeout', 'NO Timeout',
       'Blocked FG', 'Fumble Touchdown',
       'Desean Jackson 85 Yard Punt Return',
       'Brent Celek 9 Yard Pass From Donovan Mcnabb',
       'Blocked Punt Touchdown', 'Intercepted Pass Touchdown',
       'Julius Jones 62 Yard Run', 'Safety', 'Ryan Grant 3 Yard Run',
       'Blocked Punt', 'Adrian Peterson 27 Yard Run', 'Fumble, Safety',
       'Blocked FG Touchdown',
       'Cadillac Williams 8 Yard Pass From Byron Leftwich', 'BUF Timeout',
       'Frank Gore 79 Yard Run', 'Allen Rossum 67 Yard Punt Return',
       'Will Blackmon 0 Yard Kickoff Return, Tackled In Own End Zone',
       'Percy Harvin 101 Yard Kickoff Return',
       'Ellis Hobbs 0 Yard Kickoff Return, Tackled In Own End Zone',
       'Penalty on Anthony Madis

In [5]:
all_drives['uid'] = all_drives['gameId'].astype(str) +"-"+ all_drives['drive'].astype(str)

In [6]:
all_drives.sample(15)

,Unnamed: 0,away,away_score_after,away_score_before,drive,home,home_score_after,home_score_before,offense,plays,result,time,yds_gained,gameId,uid
1364,7,DEN,7,0,7,DAL,10,10,DEN,1,Knowshon Moreno 9 Yard Pass From Kyle Orton,0:00,9,291004007,291004007-7
49127,17,JAX,30,30,18,BAL,0,0,BAL,4,Fumble,1:29,29,400951579,400951579-18
27247,13,OAK,21,21,13,PIT,3,3,PIT,7,Missed FG,0:52,41,331027013,331027013-13
21693,17,PHI,17,17,18,DAL,17,17,PHI,3,Punt,1:25,7,321111021,321111021-18
28125,10,PHI,17,17,10,WSH,0,0,WSH,2,End of Half,0:07,-1,331117021,331117021-10
11297,14,BAL,21,21,15,NO,14,14,BAL,3,Punt,1:00,4,301219033,301219033-15
15378,4,CIN,7,0,4,PIT,14,14,CIN,7,Touchdown,3:23,79,311113004,311113004-4
19216,0,TEN,0,0,0,DET,3,0,DET,9,Field Goal,3:22,39,320923010,320923010-0
47963,7,PIT,7,7,7,BAL,3,3,PIT,6,Punt,3:55,23,400874583,400874583-7
11469,20,ARI,24,21,21,DAL,20,20,ARI,9,Field Goal,5:05,49,301225022,301225022-21


In [7]:
# Write a function to count drives in a given dF based on outcome
# Types: TD, FG, Turnover, Punt, End of half/game, Shenanigans
# maybe make columns for each of those types except shenanigans
def find_tds(result_list):
    '''Accepts a list of drive outcomes. 
    Return a binary list of which result in touchdowns'''
    
    is_td = []
    for outcome in [x.lower() for x in result_list]:
        td = 0
        if "touchdown" in outcome:
            if "intercept" in outcome:
                td = 0
            elif "fumble" in outcome:
                td = 0
            elif "punt" in outcome:
                td = 0
            else:
                td = 1
                
        is_td.append(td)
        
    return is_td

def find_fgs(result_list):
    '''Accepts a list of drive outcomes. 
    Return a binary list of which result in field goals'''
    
    is_fg = []
    for outcome in [x.lower() for x in result_list]:
        fg = 0
        if ("field goal" in outcome) or ("fg" in outcome):
            if "block" in outcome:
                fg = 0
            else:
                fg = 1
                
        is_fg.append(fg)
        
    return is_fg

def find_punts(result_list):
    is_punt = []
    for outcome in [x.lower() for x in result_list]:
        punt = 0
        if "punt" in outcome:
            punt = 1
        is_punt.append(punt)
        
    return is_punt
    
def find_turnovers(result_list):
    is_to = []
    for outcome in [x.lower() for x in result_list]:
        to = 0
        if "intercept" in outcome:
            to = 1
        elif "fumble" in outcome:
            to = 1
        elif "downs" in outcome:
            to = 1
        elif "safety" in outcome:
            to = 1
        is_to.append(to)
        
    return is_to
    
def find_endofhalves(result_list):
    is_eoh = []
    for outcome in [x.lower() for x in result_list]:
        eoh = 0
        if "end of half" in outcome:
            eoh = 1
        elif "end of game" in outcome:
            eoh = 1
        is_eoh.append(eoh)
        
    return is_eoh

In [8]:
all_drives['TD'] = find_tds(all_drives['result'].values)

In [9]:
all_drives['FG'] = find_fgs(all_drives['result'].values)
all_drives['punt'] = find_punts(all_drives['result'].values)
all_drives['turnover'] = find_turnovers(all_drives['result'].values)
all_drives['EoH'] = find_endofhalves(all_drives['result'].values)

In [10]:
all_drives.sample(10)

,Unnamed: 0,away,away_score_after,away_score_before,drive,home,home_score_after,home_score_before,offense,plays,result,time,yds_gained,gameId,uid,TD,FG,punt,turnover,EoH
40820,19,STL,3,3,19,ARI,24,24,ARI,5,Punt,2:52,14,400791576,400791576-19,0,0,1,0,0
3520,6,KC,7,7,6,PIT,3,3,KC,6,Punt,3:08,15,291122012,291122012-6,0,0,1,0,0
46497,3,NYJ,3,3,3,NE,0,0,NYJ,3,Punt,1:54,9,400874680,400874680-3,0,0,1,0,0
35377,1,OAK,3,0,1,SF,0,0,OAK,7,Field Goal,3:05,12,400554431,400554431-1,0,1,0,0,0
11960,3,NYJ,3,3,3,BUF,0,0,NYJ,9,Punt,3:46,41,310102020,310102020-3,0,0,1,0,0
34588,1,SEA,3,0,1,ARI,0,0,SEA,9,Field Goal,4:40,40,400554357,400554357-1,0,1,0,0,0
51307,1,HOU,0,0,1,IND,7,0,IND,7,Touchdown,4:00,92,400951751,400951751-1,1,0,0,0,0
11088,12,CIN,19,16,13,CLE,10,10,CIN,9,Field Goal,4:08,76,301219004,301219004-13,0,1,0,0,0
4824,6,TEN,14,14,6,STL,0,0,STL,6,Punt,2:39,13,291213010,291213010-6,0,0,1,0,0
23151,11,JAX,3,3,11,NYJ,0,0,JAX,4,End of Half,0:12,7,321209030,321209030-11,0,0,0,0,1


In [11]:
# Summarize a game's drives
subdf = all_drives[ all_drives['gameId'] == 400874520 ]
# Group by offense, calc what percentage of drives end in TD/FG/etc.
ratios = subdf.groupby(['offense'])[['TD','FG','punt','turnover','EoH']].mean()
np.round(ratios, decimals=3)

,TD,FG,punt,turnover,EoH
offense,,,,,
CLE,0.077,0.077,0.385,0.308,0.154
PHI,0.250,0.250,0.500,0.000,0.000


In [12]:
# drop column Unnamed: 0 and save the dataFrame for future use.
all_drives.drop(['Unnamed: 0'],axis=1,inplace=True)
all_drives.sample(5)

,away,away_score_after,away_score_before,drive,home,home_score_after,home_score_before,offense,plays,result,time,yds_gained,gameId,uid,TD,FG,punt,turnover,EoH
37686,MIA,14,14,22,NYJ,27,27,NYJ,3,Punt,2:00,-6,400761514,400761514-22,0,0,1,0,0
31644,MIA,15,15,25,KC,27,27,MIA,4,Downs,0:51,9,400554280,400554280-25,0,0,0,1,0
27862,BAL,0,0,1,CIN,0,0,CIN,6,Downs,2:35,22,331110033,331110033-1,0,0,0,1,0
6687,CAR,7,7,20,TB,20,20,CAR,1,Intercepted Pass,0:00,21,300919029,300919029-20,0,0,0,1,0
33677,SEA,30,27,22,OAK,17,17,OAK,3,Punt,1:39,6,400554385,400554385-22,0,0,1,0,0


In [13]:
all_drives.to_csv('../data/espn_drives2009-2017.csv')

In [14]:
subdf = all_drives[all_drives['offense']=='CAR']
ratios = subdf.groupby(['gameId'])[['TD','FG','punt','turnover','EoH']].mean()
np.round(ratios, decimals=3)

,TD,FG,punt,turnover,EoH
gameId,,,,,
290913029,0.067,0.067,0.267,0.533,0.067
290920001,0.200,0.200,0.200,0.200,0.200
290928006,0.091,0.000,0.545,0.273,0.091
291011029,0.182,0.091,0.273,0.273,0.182
291018027,0.364,0.000,0.273,0.273,0.000
291025029,0.125,0.062,0.500,0.250,0.062
291101022,0.231,0.154,0.462,0.000,0.154
291108018,0.182,0.182,0.273,0.364,0.000
291115029,0.364,0.000,0.364,0.000,0.182


***Get starting field position and time available for the drives***

In [15]:
# Read pbp dataFrame
pbp_df = pd.read_csv('../data/espn_parsedplays2009-2017.csv')
pbp_df.sample(5)

/home/welced12/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (15,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,downdist,detail,play_num,home,away,possession,home_score,away_score,...,week,home_win,is_parseable,is_run,is_pass,is_scramble,is_punt,is_fieldgoal,yds_gained,runpass
120719,120719,3,['1st and 10 at BAL 35'],"(8:57 - 4th) (No Huddle, Shotgun) D.Orlovsky p...",4,IND,BAL,IND,3,24,...,14,0,True,False,True,False,False,False,0,True
40523,40523,6,['2nd and 10 at WSH 27'],(2:38 - 1st) J.Campbell pass incomplete short ...,7,DAL,WSH,WSH,7,0,...,16,1,True,False,True,False,False,False,0,True
39891,39891,2,['1st and 10 at ARI 24'],(11:34 - 3rd) D.Avery right end to ARZ 21 for ...,3,STL,ARI,STL,0,17,...,16,0,True,True,False,False,False,False,3,True
215070,215070,4,['3rd and 13 at DET 40'],(5:46 - 2nd) E.Manning pass short left to A.B...,5,NYG,DET,NYG,3,3,...,16,1,True,False,True,False,False,False,14,True
67406,67406,1,['2nd and 8 at ARI 23'],(4:07 - 3rd) D.Anderson sacked at ARZ 18 for -...,2,SEA,ARI,ARI,23,10,...,10,1,True,False,True,False,False,False,-5,True


In [16]:
pbp_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'downdist', 'detail', 'play_num', 'home',
       'away', 'possession', 'home_score', 'away_score', 'drive_num', 'gameId',
       'down', 'dist', 'home_fieldpos', 'qtr', 'time_rem', 'secs_rem',
       'home_lead', 'total_score', 'adj_lead', 'OT', 'home_possession',
       'season', 'week', 'home_win', 'is_parseable', 'is_run', 'is_pass',
       'is_scramble', 'is_punt', 'is_fieldgoal', 'yds_gained', 'runpass'],
      dtype='object')

In [17]:
# Extract a few select columns for the first play of each drive
relevant_cols = ['detail','drive_num','gameId','secs_rem','home_fieldpos','home_possession']
firstplays = pbp_df.loc[ pbp_df['play_num'] == 1 ][relevant_cols]
firstplays['uid'] = firstplays['gameId'].astype(str)+"-"+firstplays['drive_num'].astype(str)
firstplays[['uid','secs_rem','home_fieldpos','home_possession','detail']].sample(5)

,uid,secs_rem,home_fieldpos,home_possession,detail
317395,400874663-16,783,-35,1,(13:03 - 4th) L.McCoy left tackle to BUF 18 f...
343850,400874656-3,3230,-17,1,(8:50 - 1st) (Shotgun) D.Prescott pass incomp...
194945,331027022-12,1825,-23,1,(0:25 - 2nd) M.Ryan pass short right to H.Dou...
30581,291206011-2,3088,15,0,(6:28 - 1st) J.Addai left tackle to IND 39 for...
342849,400874559-8,2472,0,1,(11:12 - 2nd) G.Gano kicks 64 yards from CAR 3...


In [18]:
# translate the fieldposition to offense's fieldposition
home_fieldpos = firstplays['home_fieldpos'].values
home_poss = firstplays['home_possession'].values
off_fieldpos = [0 for x in home_fieldpos]

for i, tup in enumerate(zip(home_fieldpos, home_poss)):
    (hfp, hp) = tup
    if hp == 1:
        off_fieldpos[i] = hfp
    else:
        off_fieldpos[i] = -1*hfp

firstplays['starting_fieldposition'] = off_fieldpos

In [19]:
# Get ready to merge this information into the drives dataFrame
right_df = firstplays.set_index('uid')[['secs_rem','starting_fieldposition']]

In [20]:
all_drives = pd.merge( all_drives, right_df,
                       how='left',
                       left_on='uid',
                       right_index=True)

In [21]:
all_drives.sample(5)

,away,away_score_after,away_score_before,drive,home,home_score_after,home_score_before,offense,plays,result,...,yds_gained,gameId,uid,TD,FG,punt,turnover,EoH,secs_rem,starting_fieldposition
16816,JAX,14,14,15,SD,31,31,JAX,3,Punt,...,-4,311205030,311205030-15,0,0,1,0,0,1303.0,-29.0
53450,JAX,45,38,24,HOU,7,7,JAX,4,Touchdown,...,45,400951781,400951781-24,1,0,0,0,0,753.0,5.0
22740,MIA,10,10,11,NE,17,17,NE,2,End of Half,...,-1,321202015,321202015-11,0,0,0,0,1,1825.0,-30.0
3366,IND,0,0,1,NE,0,0,NE,3,Punt,...,7,291115011,291115011-1,0,0,1,0,0,3502.0,-19.0
21937,WSH,14,7,5,PHI,3,3,WSH,6,Touchdown,...,80,321118028,321118028-5,1,0,0,0,0,2655.0,-30.0


In [22]:
# Convert time of possession to seconds
top = all_drives['time'].values
top_secs = [0 for x in top]
for i, time in enumerate(top):
    (mins,secs) = time.split(":")
    total_seconds = int(secs) + 60*int(mins)
    top_secs[i] = total_seconds
    
all_drives['time_in_secs'] = top_secs

all_drives[['time','time_in_secs']].sample(5)

,time,time_in_secs
21036,0:33,33
26081,3:32,212
16749,2:26,146
42768,0:10,10
38597,1:08,68


In [23]:
# Get time remaining in half
rem = all_drives['secs_rem'].values
rem_in_half = [x for x in rem]
for i, time in enumerate(rem):
    if time > 1800:
        rem_in_half[i] -= 1800
        
all_drives['left_in_half'] = rem_in_half

all_drives[['secs_rem','left_in_half']].sample(5)

,secs_rem,left_in_half
53923,478.0,478.0
50885,1921.0,121.0
45303,1331.0,1331.0
36869,387.0,387.0
28955,1969.0,169.0


In [24]:
all_drives.to_csv('../data/espn_drives2009-2017.csv')